### Requirements

In [1]:
%pip install torch
%pip install -q bitsandbytes sentencepiece accelerate loralib datasets peft
%pip install torch
%pip install pillow
%pip install transformers
%pip install pandas numpy matplotlib
%pip install ipywidgets ipykernel

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Device

In [2]:
import torch
import torchvision.transforms as transforms
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from PIL import Image

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    device_properties = torch.cuda.get_device_properties(0)  # Assuming you're using the first CUDA device
    print(f"Device Name: {device_properties.name}")
    print(f"Total Memory: {device_properties.total_memory / 1024**3:.2f} GB")
    print(f"Compute Capability: {device_properties.major}.{device_properties.minor}")
else:
    print("Using CPU")

Device Name: NVIDIA RTX A4000
Total Memory: 15.73 GB
Compute Capability: 8.6


### Base Model

In [4]:
checkpoint = "HuggingFaceM4/idefics-9b-instruct"
#checkpoint = "HuggingFaceM4/tiny-random-idefics"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"]
)
processor = AutoProcessor.from_pretrained(checkpoint)
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")
model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

IdeficsForVisionText2Text(
  (model): IdeficsModel(
    (embed_tokens): IdeficsDecoupledEmbedding(
      num_embeddings=32000, num_additional_embeddings=3, embedding_dim=4096, partially_freeze=False
      (additional_embedding): Embedding(3, 4096)
    )
    (vision_model): IdeficsVisionTransformer(
      (embeddings): IdeficsVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(257, 1280)
      )
      (pre_layrnorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (encoder): IdeficsVisionEncoder(
        (layers): ModuleList(
          (0-31): 32 x IdeficsVisionEncoderLayer(
            (self_attn): IdeficsVisionAttention(
              (k_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
              (v_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
              (q_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
        

#### Base Model Test

In [5]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)

In [6]:
url = "https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg"
prompts = [
    # "Instruction: provide an answer to the question. Use the image to answer.\n",
    url,
    "Question: What's on the picture? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=5)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Question: What's on the picture? Answer: A kitten. Question


### Finetune

#### Dataset for Finetune

In [15]:
dataset = (load_dataset("Swapnil949/nli_multimodal_ds", split='train')
        .train_test_split(train_size=1250, test_size=150))
train_ds = dataset["train"]
test_ds = dataset["test"]
test_ds_as_is = test_ds

In [8]:
def convert_to_rgb(image):
    # `image.convert("RGB")` would only work for .jpg images, as it creates a wrong background
    # for transparent images. The call to `alpha_composite` handles this case
    if image.mode == "RGB":
        return image

    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    alpha_composite = alpha_composite.convert("RGB")
    return alpha_composite

def ds_transforms(example_batch):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std

    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std),
    ])

    prompts = []
    for i in range(len(example_batch['description'])):
        prompts.append(
            [   
                "Instruction: Output only the letter of the block corresponding to following description, nothing else. Descriptions are from the perspective of the black figure looking forward at the table.\n",
                example_batch["scenario"][i],
                f"Description: {example_batch['description'][i]}, Answer: {example_batch['answer'][i]}\n",
            ],
        )

    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)

    inputs["labels"] = inputs["input_ids"]

    return inputs

In [9]:
train_ds.set_transform(ds_transforms)
test_ds.set_transform(ds_transforms)

#### fine-tuning config

In [10]:
model_name = checkpoint.split("/")[1]
config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj"],
    lora_dropout = 0.05,
    bias="none"
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 19,750,912 || all params: 8,949,438,736 || trainable%: 0.22069442098698333


In [11]:
training_args = TrainingArguments(
    output_dir = f"{model_name}-nli-multimodal",
    learning_rate = 2e-4,
    fp16 = True,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 8,
    dataloader_pin_memory = False,
    save_total_limit = 3,
    evaluation_strategy ="steps",
    save_strategy = "steps",
    eval_steps = 10,
    save_steps = 25,
    max_steps = 50,
    logging_steps = 5,
    remove_unused_columns = False,
    push_to_hub=False,
    label_names = ["labels"],
    load_best_model_at_end = False,
    report_to = "none",
    optim = "paged_adamw_8bit",
)

In [12]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ds,
    eval_dataset = test_ds
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


#### train

In [13]:
trainer.train()

Step,Training Loss,Validation Loss
10,1.903700,1.046834
20,0.508300,0.518584
30,0.513300,0.487497
40,0.425200,0.476575
50,0.525600,0.473377


TrainOutput(global_step=50, training_loss=0.9708874797821045, metrics={'train_runtime': 437.6302, 'train_samples_per_second': 0.914, 'train_steps_per_second': 0.114, 'total_flos': 1316672293766208.0, 'train_loss': 0.9708874797821045, 'epoch': 0.32})

#### Test

In [14]:
prompts = []
prompts.append(
    [   
        "Instruction: Output only the letter of the block corresponding to following description, nothing else. Descriptions are from the perspective of the black figure looking forward at the table.\n",
        testest_ds_as_ist_ds["scenario"][10],
        f"Description: {test_ds_as_is['description'][10]}, Answer:\n",
    ],
)
check_inference(model, processor, prompts, max_new_tokens=100)

KeyError: 'description'